In [23]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt 
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool
from catboost.utils import get_roc_curve
import sklearn

pd.pandas.set_option('display.max_columns', None)

In [24]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [25]:
train['BANKR'].value_counts()


0    730141
1      3130
Name: BANKR, dtype: int64

In [26]:
train = train.sort_values('BANKR')
train.head(5)

,ID_FIRM,P1110_B,P1110_E,P1120_B,P1120_E,P1130_B,P1130_E,P1140_B,P1140_E,P1150_B,P1150_E,P1160_B,P1160_E,P1170_B,P1170_E,P1180_B,P1180_E,P1190_B,P1190_E,P1100_B,P1100_E,P1210_B,P1210_E,P1220_B,P1220_E,P1230_B,P1230_E,P1240_B,P1240_E,P1250_B,P1250_E,P1260_B,P1260_E,P1200_B,P1200_E,P1600_B,P1600_E,P1310_B,P1310_E,P1320_B,P1320_E,P1340_B,P1340_E,P1350_B,P1350_E,P1360_B,P1360_E,P1370_B,P1370_E,P1300_B,P1300_E,P1410_B,P1410_E,P1420_B,P1420_E,P1430_B,P1430_E,P1450_B,P1450_E,P1400_B,P1400_E,P1510_B,P1510_E,P1520_B,P1520_E,P1530_B,P1530_E,P1540_B,P1540_E,P1550_B,P1550_E,P1500_B,P1500_E,P1700_B,P1700_E,P2110_B,P2110_E,P2120_B,P2120_E,P2100_B,P2100_E,P2210_B,P2210_E,P2220_B,P2220_E,P2200_B,P2200_E,P2310_B,P2310_E,P2320_B,P2320_E,P2330_B,P2330_E,P2340_B,P2340_E,P2350_B,P2350_E,P2300_B,P2300_E,P2410_B,P2410_E,P2421_B,P2421_E,P2430_B,P2430_E,P2450_B,P2450_E,P2460_B,P2460_E,P2400_B,P2400_E,BANKR
0,1,8,7,0,0,0,0,0,0,402,398.0,0,0,115,96,0,0,0,0,525,501.0,2,2,0,0,3640.0,2174.0,28672,26515,2117,1694,89,155,34520.0,30540,35045,31041,785.0,785.0,0,0,0,0,13770,13770,114,114,19949,16589.0,34618,31258,0,0,0,0,0,0,0,0.0,0,0.0,0.0,0,824,5.0,0,0,0,0,0,0,824,5,35442.0,31263.0,14702.0,14581.0,0,0.0,14702,14581,5256,5513,24917,23451,-15471,-14383.0,19246,17024,2699,2517,0,0,49,3,4884,4460.0,1639,701,78,80,345,161,0,0,0,0,0,0,1561,621,0
488674,488675,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,82,82,0,0,82,82.0,13241,7548,0,119,81533.0,87271.0,0,0,7157,810,0,0,101931.0,95748,102013,95830,10.0,10.0,0,0,0,0,0,0,0,0,1038,1453.0,1048,1463,0,0,0,0,0,0,0,0.0,0,0.0,0.0,0,101513,95050.0,0,0,0,0,0,0,101513,95050,102561.0,96513.0,197949.0,164198.0,183792,160241.0,14157,3957,5619,4644,1132,1129,7406,-1816.0,0,0,0,0,0,0,16228,8859,15757,8933.0,7877,-1890,168,201,0,0,0,0,0,0,0,99,7709,-2190,0
488675,488676,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,218,390,0,0,218,390.0,141,0,0,0,785.0,2125.0,0,0,376,244,0,0,1302.0,2369,1520,2759,10.0,10.0,0,0,0,0,0,0,0,0,-885,-1521.0,-875,-1511,0,2236,0,0,0,0,0,0.0,0,2236.0,0.0,0,2398,2108.0,0,0,0,0,0,0,2398,2108,1523.0,2833.0,10986.0,14685.0,10805,15715.0,181,-1030,0,0,0,0,181,-1030.0,0,0,0,0,0,0,0,0,53,56.0,128,-1086,22,0,0,0,0,0,0,0,0,0,106,-1086,0
488676,488677,0,0,0,0,0,0,0,0,153,74.0,0,0,0,0,0,0,0,0,153,74.0,3049,3172,0,0,1862.0,3865.0,0,0,632,4389,133,127,5676.0,11553,5829,11627,13.0,13.0,0,0,0,0,0,0,0,0,-5591,-5360.0,-5578,-5347,0,0,0,0,0,0,0,0.0,0,0.0,5.0,5,11274,16796.0,0,0,0,608,0,0,11279,17409,5701.0,12062.0,112049.0,111150.0,105555,102749.0,6494,8401,0,0,8040,13058,-1546,-4657.0,0,0,0,0,0,0,134,10,266,153.0,-1678,-4800,76,42,0,-1,0,0,0,0,0,0,-1754,-4842,0
488677,488678,0,0,0,0,0,0,0,0,130,96.0,0,0,0,0,0,0,0,0,130,96.0,0,2,0,0,13654.0,11293.0,0,0,22,1580,0,0,13676.0,12875,13806,12971,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,1304.0,0,11795,11539.0,0,0,0,0,0,0,13099,11539,13099.0,11539.0,74446.0,118848.0,73746,122461.0,700,-3613,0,0,0,0,700,-3613.0,0,0,0,0,125,40,0,0,181,122.0,394,-3775,31,113,0,0,0,0,0,0,0,0,363,-3888,0


In [27]:
small_train = train[-1*int(3130*2.5):]

In [28]:
small_train['BANKR'].value_counts()

0    4695
1    3130
Name: BANKR, dtype: int64

In [29]:
X_small = small_train.drop(["BANKR", "ID_FIRM"], axis=1).fillna(0)
Y_small = small_train['BANKR']

In [30]:
X_train, X_val, y_train, y_val = train_test_split(X_small, Y_small, train_size=0.8, random_state=42)

In [31]:
model=CatBoostClassifier(
    eval_metric='BalancedAccuracy',
    # scale_pos_weight=233.27,
    iterations=300,
    learning_rate=0.3,
    # early_stopping_rounds=20,
    random_seed=63)

In [32]:
model2=CatBoostClassifier()

In [33]:
sel_feat =[]
for feature in X_train:
    sel_feat.append(feature)

In [34]:
model.select_features(
    X=X_train,
    y=y_train,
    features_for_select=sel_feat,
    num_features_to_select=100,
    eval_set=(X_val, y_val),
    plot=True, 
    train_final_model=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Step #1 out of 1
0:	learn: 0.7928856	test: 0.7935125	best: 0.7935125 (0)	total: 13.8ms	remaining: 4.14s
1:	learn: 0.7985577	test: 0.7955845	best: 0.7955845 (1)	total: 19.5ms	remaining: 2.91s
2:	learn: 0.8024891	test: 0.7974359	best: 0.7974359 (2)	total: 24.5ms	remaining: 2.42s
3:	learn: 0.8122674	test: 0.8059540	best: 0.8059540 (3)	total: 31ms	remaining: 2.29s
4:	learn: 0.8210500	test: 0.8078437	best: 0.8078437 (4)	total: 35.8ms	remaining: 2.11s
5:	learn: 0.8235781	test: 0.8123745	best: 0.8123745 (5)	total: 40.7ms	remaining: 1.99s
6:	learn: 0.8254425	test: 0.8163746	best: 0.8163746 (6)	total: 47.1ms	remaining: 1.97s
7:	learn: 0.8273060	test: 0.8235977	best: 0.8235977 (7)	total: 53.9ms	remaining: 1.97s
8:	learn: 0.8280376	test: 0.8195849	best: 0.8235977 (7)	total: 61.4ms	remaining: 1.99s
9:	learn: 0.8334257	test: 0.8217208	best: 0.8235977 (7)	total: 66.4ms	remaining: 1.93s
10:	learn: 0.8388784	test: 0.8233259	best: 0.8235977 (7)	total: 71.4ms	remaining: 1.87s
11:	learn: 0.8397432	test: 

{'selected_features': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  26,
  27,
  28,
  29,
  30,
  32,
  33,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  82,
  83,
  84,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109],
 'eliminated_features_names': ['P1600_B',
  'P1230_E',
  'P1180_B',
  'P1300_B',
  'P1230_B',
  'P2200_E',
  'P1510_B',
  'P2210_E',
  'P1260_E',
  'P2340_E'],
 'loss_graph': {'main_indices': [0, 10],
  'removed_features_count': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'loss_values': [0.35980840763729915,
   0.3598559535388351,
   0.3598897825291733,
   0.3599117062112423,
   0.

In [35]:
y_pred = model.predict(X_val)
sklearn.metrics.roc_auc_score(y_val, y_pred)

0.8514158257592023

In [36]:
model2.fit(X_train,
           y_train,
           verbose=False,
           eval_set=(X_val, y_val),
           plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [37]:
y_pred1 = model.predict(test)
y_pred2 = model2.predict(test)

In [38]:
len(y_pred1)

314259

In [39]:
len(y_pred2)

314259

In [41]:
y_pred1 = model.predict(test)
cnt1 = 0
cnt0 = 0
for y in y_pred:
  if y == 0: cnt0+=1
  else: cnt1+=1

print(cnt0, cnt1)

961 604


In [42]:
submission_df = pd.DataFrame({'ID_FIRM': test['ID_FIRM'], 'BANKR': y_pred1})
submission_df.to_csv('submission1.csv', index=False)

submission_df = pd.DataFrame({'ID_FIRM': test['ID_FIRM'], 'BANKR': y_pred2})
submission_df.to_csv('submission2.csv', index=False)